<a href="https://colab.research.google.com/github/thomas-greig/MSc/blob/main/NR15Exponent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

# =========================
# Global Parameters
# =========================
Lx, Ly = 40, 40
N = 800
steps = 15_000_000
b_over_T = 4.0           # β/T
J_over_T = b_over_T/3.0  # J/T
eps = 0.8
drift_direction = (1, 0)
sample_every = 800
relaxation_index = int(0.2 * steps / sample_every)

# Toggle plotting for the final run only (to keep output sane)
DO_PLOTS = True

# =========================
# Helpers
# =========================
nn_disps = [(-1,0),(1,0),(0,-1),(0,1)]

def nn_sum(occ, x, y):
    s = 0
    for dx,dy in nn_disps:
        s += occ[(x+dx) % Lx, (y+dy) % Ly]
    return s

def compute_2D_move_probabilities(positions, idx, occupancy, Lx, Ly, b_over_T, J_over_T, eps, drift_direction):
    moves = [(-1, 0), (1, 0), (0, 0), (0, 1), (0, -1)]  # L, R, S, U, D
    weights = []

    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(x))

    dx_bias, dy_bias = drift_direction
    x_old, y_old = positions[idx]

    n_old = occupancy[x_old, y_old]
    S_old = nn_sum(occupancy, x_old, y_old)

    for dx, dy in moves:
        if dx == 0 and dy == 0:
            dE = 0.0
        else:
            x_trial = (x_old + dx) % Lx
            y_trial = (y_old + dy) % Ly

            n_trial = occupancy[x_trial, y_trial]
            S_trial = nn_sum(occupancy, x_trial, y_trial)

            dE_on = 2.0 * b_over_T * (1 + n_trial - n_old)
            dE_nn = - J_over_T * (S_trial - S_old - 1)
            drive = -(dx * dx_bias + dy * dy_bias)

            dE = dE_on + dE_nn + drive

        base_prob = sigmoid(dE)
        drift_alignment = dx * dx_bias + dy * dy_bias  # 0 for stay
        weight = (1 + eps * drift_alignment) * base_prob
        weights.append(weight)

    weights = np.asarray(weights, dtype=float)
    probs = weights / np.sum(weights)
    return probs, moves

def compute_2D_correlation(data):
    from numpy.fft import fft2, ifft2, fftshift
    T, Lx_, Ly_ = data.shape
    mean_n = data.mean(axis=0)
    fluct = data - mean_n
    corr_fft = np.zeros((Lx_, Ly_), dtype=np.complex128)
    for frame in fluct:
        f = fft2(frame)
        corr_fft += f * np.conj(f)
    corr = np.real(ifft2(corr_fft)) / (T * Lx_ * Ly_)
    corr = fftshift(corr)
    return corr

def plot_directional_cuts(corr, log_log=False):
    Lx_, Ly_ = corr.shape
    cx, cy = Lx_ // 2, Ly_ // 2
    cut_x = corr[cx:, cy];  r_x = np.arange(0, len(cut_x))
    cut_y = corr[cx, cy:];  r_y = np.arange(0, len(cut_y))
    plt.figure()
    plt.plot(r_x[1:], cut_x[1:], 'o-', label='Along x-axis (dy=0)')
    plt.plot(r_y[1:], cut_y[1:], 's-', label='Along y-axis (dx=0)')
    if log_log:
        plt.xscale('log')
        plt.yscale('log')
        plt.grid(True, which="both", ls="-", alpha=0.5)
    else:
        plt.yscale('log')
        plt.grid(True, which="both", ls="-", alpha=0.5)
    plt.xlabel('r (lattice units)'); plt.ylabel('C(r)')
    plt.legend(); plt.show()

# =========================
# One full simulation + fits
# =========================
def simulate_and_fit(seed=None, do_plots=False):
    if seed is not None:
        np.random.seed(seed)

    # --- Initialise randomly ---
    positions = np.column_stack((
        np.random.randint(0, Lx, size=N),
        np.random.randint(0, Ly, size=N)
    ))
    occupancy = np.zeros((Lx, Ly), dtype=int)
    for x, y in positions:
        occupancy[x, y] += 1

    # --- Run Simulation ---
    n_samples = steps // sample_every + 1
    occupancy_time_series = np.zeros((n_samples, Lx, Ly), dtype=int)
    sample_index = 0

    for t in range(steps):
        idx = np.random.randint(N)
        probs, moves = compute_2D_move_probabilities(
            positions, idx, occupancy, Lx, Ly, b_over_T, J_over_T, eps, drift_direction
        )
        dx, dy = moves[np.random.choice(len(moves), p=probs)]
        x_old, y_old = positions[idx]
        x_new = (x_old + dx) % Lx
        y_new = (y_old + dy) % Ly
        positions[idx] = [x_new, y_new]
        occupancy[x_old, y_old] -= 1
        occupancy[x_new, y_new] += 1

        if t % sample_every == 0:
            occupancy_time_series[sample_index] = occupancy
            sample_index += 1

    # --- Correlation ---
    stationary = occupancy_time_series[relaxation_index:]
    corr_2d = compute_2D_correlation(stationary)

    # --- Directional cuts and log-log fits (all positive) ---
    Lx_, Ly_ = corr_2d.shape
    cx, cy = Lx_ // 2, Ly_ // 2
    cut_x = corr_2d[cx:, cy]
    r_x = np.arange(0, len(cut_x))
    cut_y = corr_2d[cx, cy:]
    r_y = np.arange(0, len(cut_y))

    # masks to avoid non-positive values
    pos_x = cut_x[1:] > 0
    pos_y = cut_y[1:] > 0

    slope_x = np.nan
    slope_y = np.nan
    r2_x = np.nan
    r2_y = np.nan

    if np.count_nonzero(pos_x) >= 2:
        log_r_x = np.log(r_x[1:][pos_x])
        log_cut_x = np.log(cut_x[1:][pos_x])
        slope_x, intercept_x, r_value_x, _, _ = linregress(log_r_x, log_cut_x)
        r2_x = r_value_x**2

    if np.count_nonzero(pos_y) >= 2:
        log_r_y = np.log(r_y[1:][pos_y])
        log_cut_y = np.log(cut_y[1:][pos_y])
        slope_y, intercept_y, r_value_y, _, _ = linregress(log_r_y, log_cut_y)
        r2_y = r_value_y**2

    # --- Restricted fitting range on x-axis data (example) ---
    # Use same x-axis positives, then window 1 < r < 15 (adjust as needed)
    exponent_range = np.nan
    if np.count_nonzero(pos_x) >= 2:
        r_x_full = r_x[1:][pos_x]
        cut_x_full = cut_x[1:][pos_x]
        fit_mask = (r_x_full > 1) & (r_x_full < 15)
        if np.count_nonzero(fit_mask) >= 2:
            log_r = np.log(r_x_full[fit_mask])
            log_C = np.log(cut_x_full[fit_mask])
            slope, intercept = np.polyfit(log_r, log_C, 1)
            exponent_range = slope

            if do_plots:
                # Optional visuals for the last run
                mean_total = occupancy_time_series.mean(axis=0)
                mean_stationary = occupancy_time_series[relaxation_index:].mean(axis=0)

                plt.imshow(mean_total.T, origin='lower', cmap='viridis'); plt.colorbar(label='Mean Occupancy')
                plt.xlabel("x"); plt.ylabel("y"); plt.show()

                plt.imshow(mean_stationary.T, origin='lower', cmap='viridis'); plt.colorbar(label='Mean Occupancy')
                plt.xlabel("x"); plt.ylabel("y"); plt.show()

                plt.imshow(corr_2d.T, origin='lower', cmap='coolwarm'); plt.colorbar(label='C(dx, dy)')
                plt.xlabel("dx"); plt.ylabel("dy"); plt.show()

                plot_directional_cuts(corr_2d)
                plot_directional_cuts(corr_2d, log_log=True)

                # Log-log scatter + fit line for the restricted fit
                plt.figure()
                plt.loglog(r_x_full, cut_x_full, 'o', label="x-axis data")
                r_line = np.linspace(np.min(r_x_full[fit_mask]), np.max(r_x_full[fit_mask]), 100)
                C_line = np.exp(intercept) * r_line**slope
                plt.loglog(r_line, C_line, '-', label=f"fit (1<r<15): exponent={slope:.2f}")
                plt.xlabel("r (lattice units)")
                plt.ylabel("C(r)")
                plt.legend()
                plt.grid(True, which="both", ls="-", alpha=0.5)
                plt.show()

    return {
        "slope_x": slope_x,
        "slope_y": slope_y,
        "exponent_range": exponent_range,
        "r2_x": r2_x,
        "r2_y": r2_y
    }

# =========================
# Run X times and aggregate
# =========================
runs = 1
base_seed = 52341  # change if you like reproducibility

results = []
for i in range(runs):
    do_plot_this = DO_PLOTS and (i == runs - 1)
    res = simulate_and_fit(seed=base_seed + i, do_plots=do_plot_this)
    results.append(res)

exponents_x = np.array([r["slope_x"] for r in results], dtype=float)
exponents_y = np.array([r["slope_y"] for r in results], dtype=float)
exponents_range = np.array([r["exponent_range"] for r in results], dtype=float)
r2_x_all = np.array([r["r2_x"] for r in results], dtype=float)
r2_y_all = np.array([r["r2_y"] for r in results], dtype=float)

def finite(arr):
    return arr[np.isfinite(arr)]

def stats(arr):
    a = finite(arr)
    if a.size == 0:
        return np.nan, np.nan
    return np.mean(a), np.std(a, ddof=1) if a.size > 1 else 0.0

mean_x, std_x = stats(exponents_x)
mean_y, std_y = stats(exponents_y)
mean_rng, std_rng = stats(exponents_range)
mean_r2x, _ = stats(r2_x_all)
mean_r2y, _ = stats(r2_y_all)

# =========================
# Report
# =========================
np.set_printoptions(precision=4, suppress=True)

print("Power-law exponents (log-log linear fit over all positive points):")
print(f"  X-axis slopes (10 runs): {exponents_x}")
print(f"  Y-axis slopes (10 runs): {exponents_y}")
print(f"  Mean±SD X: {mean_x:.4f} ± {std_x:.4f}")
print(f"  Mean±SD Y: {mean_y:.4f} ± {std_y:.4f}")
print(f"  Mean R^2 X: {mean_r2x:.4f} | Mean R^2 Y: {mean_r2y:.4f}")

print("\nRestricted-range fit on x-axis (1 < r < 15):")
print(f"  Exponents (10 runs): {exponents_range}")
print(f"  Mean±SD: {mean_rng:.4f} ± {std_rng:.4f}")